In [1]:
# import packages
import pandas as pd
import pyarrow.dataset as ds
from datasets import Dataset
import pyarrow as pa
from datasets import *
from transformers import *
from tokenizers import *
import os
import json
import sys
from tokenizers.processors import BertProcessing
import numpy as np

C:\Users\XinZhou\anaconda3\envs\dna\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\XinZhou\anaconda3\envs\dna\lib\site-packages\torchaudio\backend\utils.py:62: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [17]:
# loading for different datasets

MAX_LENGTH = 512
batch_size = 8
num_epochs = 5
max_step_saves = 3
test_only = False
data_name = 'devign' # poj, devign, draper, d2a, mvdsc, sysevr
truncate_longer_samples = True
# is_windows = sys.platform.startswith('win')
# if is_windows:

# data_dir = "..\\..\\cbert\\dna_data_rename_tag"
data_dir = "..\\..\\cbert\\test_data"
out_dir = 'result_{}'.format(data_name)

# bert_model_path = '..\\..\\cbert\\pretrained\\test_albert_NaCUT\\checkpoint'
bert_model_path = '..\\pretrained-dna-bert\\best'
tokenizer_path = '..\\WordPiece_tokenizer'


In [18]:
import pickle 


def preprocess(text):
    if not text:
        return 'None'
    for s in ['(', ')', ':']:
        text = text.split(s)
        replace = ' {} '.format(s)
        text = replace.join(text)
    text = ' '.join(text.split())
    return text


def load_DNa_data(filename, mode='code'):
    df = {'filename': [], 'text': [], 'label': []}
    with open(filename, 'rb') as f:
        mydict = pickle.load(f)
        #print('number of rows:', len(mydict['filename']))
        for i in range(len(mydict['filename'][:1000])):
            text = mydict[mode][i]
            text = preprocess(text)
            df['filename'].append(mydict['filename'][i])
            df['text'].append(text)
            df['label'].append(mydict['label'][i])
    print('done...')
    df = pd.DataFrame(df)
    ### convert to Huggingface dataset
    hg_dataset = Dataset(pa.Table.from_pandas(df))
    return hg_dataset

# train_data = get_data('/scratch/xin/bert_source/mvdsc/tokenized_mvdsc_train.pkl')
# valid_data = get_data('/scratch/xin/bert_source/mvdsc/tokenized_mvdsc_valid.pkl')
if data_name == 'devign':
    raw_data_path = os.path.join(data_dir, data_name, 'devign_all.pkl')
    source_code_data = load_DNa_data(raw_data_path, mode='code')
    data = source_code_data.train_test_split(test_size=0.2)
    train_data = data['train']
    other_data = data['test']
    split_data = other_data.train_test_split(test_size=0.5)
    valid_data = split_data['train']
    test_data = split_data['test']

    
elif data_name == 'mvdsc':
    raw_data_path = os.path.join(data_dir, data_name, 'mvdsc_train.pkl')
    train_data = load_DNa_data(raw_data_path, mode='code')
    raw_data_path = os.path.join(data_dir, data_name, 'mvdsc_valid.pkl')
    valid_data = load_DNa_data(raw_data_path, mode='code')
    raw_data_path = os.path.join(data_dir, data_name, 'mvdsc_test.pkl')
    test_data = load_DNa_data(raw_data_path, mode='code')



# train_data = train_data[:100]
# valid_data = valid_data[:100]
# test_data = test_data[:100]
print(train_data)
print(valid_data)
print(test_data)



done...
Dataset({
    features: ['filename', 'text', 'label'],
    num_rows: 800
})
Dataset({
    features: ['filename', 'text', 'label'],
    num_rows: 100
})
Dataset({
    features: ['filename', 'text', 'label'],
    num_rows: 100
})


In [19]:
from transformers import AutoTokenizer
import transformers
print(tokenizer_path)
# tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
print(tokenizer_path)
tokenizer = transformers.AutoTokenizer.from_pretrained(tokenizer_path)

def preprocess_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True, max_length=MAX_LENGTH)



Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file ..\WordPiece_tokenizer\config.json
Model config BertConfig {
  "BOS_token": "<s>",
  "EOS": "</s>",
  "_name_or_path": "..\\WordPiece_tokenizer",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "cls_token": "[CLS]",
  "do_lower_case": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "mask_token": "[MASK]",
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token": "[PAD]",
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "unk_token": "[UNK]",
  "use_cache": true,
  "vocab_size": 30522
}

Didn't find file ..\WordPiece_tokenizer\tokenizer.json. We won't load it.
Didn't find file ..\Word

..\WordPiece_tokenizer
..\WordPiece_tokenizer


In [20]:

tokenized_train_data = train_data.map(preprocess_function, batched=True)
tokenized_valid_data = valid_data.map(preprocess_function, batched=True)
tokenized_test_data = test_data.map(preprocess_function, batched=True)

print(tokenized_train_data)
print(tokenized_valid_data)
print(tokenized_test_data)
print(len(tokenized_train_data[1]['input_ids']))

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# from torch.utils.data import DataLoader
# train_dataloader = DataLoader(train_data, shuffle=True, batch_size=8)
# valid_dataloader = DataLoader(valid_data, batch_size=8)
# test_dataloader = DataLoader(test_data, batch_size=8)







100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 11.63ba/s]

Dataset({
    features: ['filename', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 800
})
Dataset({
    features: ['filename', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 100
})
Dataset({
    features: ['filename', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 100
})
512


In [21]:
# from transformers import DataCollatorWithPadding

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# from transformers import AutoModelForSequenceClassification

# model = AutoModelForSequenceClassification.from_pretrained(bert_model_path, num_labels=2)

# import wandb

# wandb.init(project="test-project", entity="xinzhou")
# # wandb.config = {
# #   "learning_rate": 0.001,
# #   "epochs": 100,
# #   "batch_size": 128
# # }
# # wandb.log({"loss": loss})

# # Optional
# # wandb.watch(model)

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(bert_model_path, num_labels=2)



loading configuration file ..\pretrained-dna-bert\best\config.json
Model config BertConfig {
  "_name_or_path": "..\\pretrained-dna-bert\\best",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 8929
}

loading weights file ..\pretrained-dna-bert\best\pytorch_model.bin
Some weights of the model checkpoint at ..\pretrained-dna-bert\best were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNor

In [ ]:
from transformers import TrainingArguments, Trainer
output_dir="result_{}".format(data_name)


batch_size_per_gpu = 8
save_step = 100

training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True, 
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size_per_gpu,
    per_device_eval_batch_size=batch_size_per_gpu,
    gradient_accumulation_steps=8, # accumulating the gradients before updating the weights
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=save_step,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=save_step,
    load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    save_total_limit=5,           # whether you don't have much space so you let only 3 model weights saved in the disk
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_valid_data,
    data_collator=data_collator,
)

trainer.train()

PATH = os.path.join(output_dir, 'checkpoint-best')

trainer.save_model(PATH)


using `logging_steps` to initialize `eval_steps` to 100
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: filename, text. If filename, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 800
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 8
  Total optimization steps = 36
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss


In [15]:

predict_output = trainer.predict(test_dataset=tokenized_test_data)



The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: filename, text. If filename, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 10
  Batch size = 8


In [16]:
import numpy as np
# predictions, label_ids, metrics
predictions = predict_output[0]
labels = predict_output[1]

# print(type(predictions), predictions.shape)
predicted_values = np.argmax(predictions, 1)

print(predicted_values[:100])
print(labels[:100])
test_accuracy = np.sum(predicted_values == labels) / len(labels)

print ("Test Accuracy:", test_accuracy)

[0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 1 0 1 1 1 0]
Test Accuracy: 0.5


In [12]:
# import itertools

# # plot confusion matrix
# # code borrowed from scikit-learn.org
# def plot_confusion_matrix(cm, classes,
#                           normalize=False,
#                           title='Confusion matrix',
#                           cmap=plt.cm.Blues):
#     """
#     This function prints and plots the confusion matrix.
#     Normalization can be applied by setting `normalize=True`.
#     """
#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     print(cm)

#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)

#     fmt = '.2f' if normalize else 'd'
#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, format(cm[i, j], fmt),
#                  horizontalalignment="center",
#                  color="white" if cm[i, j] > thresh else "black")

#     plt.tight_layout()
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')

NameError: name 'plt' is not defined

In [11]:
# from sklearn.metrics import classification_report, confusion_matrix

# cm_test = confusion_matrix(true_values, predicted_values)

# np.set_printoptions(precision=2)

# plt.figure(figsize=(6,6))
# plot_confusion_matrix(cm_test, classes=label_values, title='Confusion Matrix - Test Dataset')
# plt.figure(figsize=(6,6))
# plot_confusion_matrix(cm_test, classes=label_values, title='Confusion Matrix - Test Dataset', normalize=True)